* 二分

二分法的基础用法是在单调序列或单调函数中查找。当问题的答案具有单调性时，就可以通过二分把求解转化为判定。**二分的实现方法多种多样，细节很多，只有10%的程序员能写对二分法**

1. 整数集合上的二分 **这个很重要**

下面根据两种情况，有两种写法，保证答案处于闭区间[l,r]以内，循环以l=r结束，每次二分的中间值 mid 会归属左半段与右半段两者之一。

In [ ]:
# 在单调递增序列a中查找 >= x 的数中最小的一个（即x或者x的后继）
while l < r :
    mid = (l+r) >> 1
    if a[mid] >= x:     # 根据a的单调性，后面的数会更大
        r = mid
    else:               # a[mid] < x,所以要将左侧下标标为 mid+1
        l = mid + 1 
# 在单调递增序列a中查找 <= x 的数中最大的一个（即x或者x的前驱）
while l < r:
    mid = (l+r+1) >> 1  # 这里如果不+1，当r-l=1时，(l+r)>>1 = l，如果进入if分支就会死循环
    if a[mid] <= x:     # 同上
        l = mid
    else:               # 同上
        r = mid - 1

"""
再分析：
第一段代码里面的 mid = (l+r) >> 1，发现永远不会取到r这个值
第二段代码里面的 mid = (l+r+1) >> 1，发现永远不会取到l这个值
这样的话：
对于第一段代码，扩大二分区间由 [1,n] --> [1,n+1]，如果最后二分终止于扩大后的这个越界下标 n+1 上，则说明a中不存在所求的数
对于第二段代码，扩大二分区间由 [1,n] --> [0,n]， 如果最后二分终止于扩大后的这个越界下标 0 上，则说明a中不存在所求的数
"""

* 总结：
1. 对于求后继，缩小范围时 $r=mid,l=mid+1$，取中间值时，$mid=(l+r)>>1$。
2. 对于求前驱，缩小范围时 $l=mid,r=mid-1$，取中间值时，$mid=(l+r=1)>>1$。
3. 根据具体情况选择1和2不同的写法
4. 二分终止条件是 l==r，就是答案所在位置

2. 实数域上的二分

确定好所需的精度 eps，以 l+eps < r 为循环条件，每次根据在mid上的判定选择r = mid 或 l = mid 分支之一即可。一般需要保留k位小数时，则取 $eps=10^{-(k+2)}$。如果不确定精度，可直接使用固定次数的二分方法

In [ ]:
# 使用精度作为循环条件的写法
while l + 1e-5 < r:
    mid = (l+r) / 2
    if calc(mid):
        r = mid
    else:
        l = mid

# 使用次数作为循环的写法
for i in range(100):
    mid = (l+r)/2
    if calc(mid):
        r = mid
    else:
        l = mid

3. 三分求单峰函数极值
* 定义
  * 单峰函数：拥有唯一的极大值点
  * 单谷函数：拥有唯一的极小值点
* 以单峰函数为例子，函数定义在[l,r]上，任取两个点 lmid,rmid，把函数分成三段
  * 若 f(lmid) < f(rmid)，令 l = lmid
  * 若 f(lmid) > f(rmid)，令 r = rmid
  * 若 f(lmid) = f(rmid)，令 l = lmid, r = rmid

4. 二分答案转化为判定

对于很多问题，我们很难直接求一个问题的解，但是我们可以对这些可行方案进行评估得到的数值构成函数的"值域"中的值进行判断，判断值域中是否存在一个值满足解。将求解问题 --> 判定问题。这样的话，我们可以通过二分的方法，将值域不断划分，从而得到满足条件的解。

* 特点    
如果一个题目出现了 **最大值最小** 的含义，这是答案具有单调性，可用二分转化为判定的最常见、最典型的特征之一。

4.1 例题    
N本书排成一行，已知第$i$本的厚度是$A_i$。把它们分成**连续的M**组，使T最小化，其中T表示厚度之和最大的一组的厚度。

* 题解：分成两步
  * 假设答案是S
  * 使用贪心算法去装书
    * 如果装完后，< M 组，那我们要搜索S以上的值域
    * 如果装完后，> M 组，那我们要搜索S以下的值域
  * 使用二分法来确定S，直到找到合适的S

In [ ]:
def valid(S):
    group = 1,rest = S
    for i in range(n):
        if rest >= a[i]:        # 如果能装下就装在这个组
            rest-=a[i]
        else:                   # 否则装在一个新的组
            group +=1
            rest = S-a[i]
    return group <= m
def main():
    l,r = 0,sum(a)
    while l < r:                # 显然这里是求后继
        mid = (l+r) / 2
        if valid(mid):
            r = mid
        else:
            l = mid + 1
    print(l)

4.2 Best Cow Fences  
给定正整数数列A，求一个平均数最大的、长度不小于L的（连续的）子段。

* 解题步骤：
  1. 先转为判定问题：是否存在一个长度不小于L的子段，平均数**不小于**二分的值
  2. 把数列值都**减去平均数**，问题转为判定**是否存在一个长度不小于L的子段，子段和非负**
  3. 设 $sum_i$ 表示 $A_1 ~ A_i$。则**子段和**可以表示为 $\max_{i-j \geqslant L} \{A_{j+1}+A_{j+2}+...+A_i\} = \max_{L \leqslant i \leqslant n} \{sum_i - \min_{0 \leqslant i-L}\{sum_j\}\}$
  4. 观察上式发现，我们只要求出0~i-L的最小子段和，就可以判断步骤2是否存在

4.3 Innovative Business

有N个元素，每个元素之间的大小关系是确定的，关系不具有传递性。也就是说，元素的大小关系是N个点与N*(N-1)/2条有向边构成的任意有向图。    
然而，这是一道交互式试题，这些关系不能一次性得知，你必须通过不超过10000次提问，每次提问只能了解某两个元素之间的关系，把这N个元素排成一行，使得每个元素都小于右边与它相邻的元素， $ N \leqslant 1000$ 。

* 解题思路：
  * 仔细读题，发现，这就是构建二叉查找树的过程
  * 我们假设前k-1个元素已经按照要求排成一行，通过二分查找确定第k个元素应该放在哪个前面 （这里使用后继和前驱都行）